<a href="https://colab.research.google.com/github/liz-lewis-manchester/CNM_2025_group_01/blob/testing-solver/Test_case_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation
from IPython.display import HTML

class AdvectionSolver:
  #Finite difference solver for the 1D advection equation: d(theta)/dt + U*d(theta)/dx = 0


    def __init__(self, L, dx, dt, T, U):
        self.L = L
        self.dx = dx
        self.dt = dt
        self.T = T
        self.U = U

        # Spatial grid
        self.x_new = np.arange(0, L+dx, dx)
        self.num_points = len(self.x_new)
        self.num_time_steps = int(self.T / self.dt)

        if self.U <= 0:
            raise ValueError("This solver assumes flow velocity U > 0.")

    def _solve_implicit(self, theta_initial, boundary_theta):
        """
        Implicit Upwind Scheme (Standard BTBS).
        Unconditionally stable.
        Equation: theta_i^{n+1} = (theta_i^n + C * theta_{i-1}^{n+1}) / (1 + C)
        """
        print(f"Using **Implicit Scheme (Standard BTBS)**.")

        theta = np.copy(theta_initial)
        N = self.num_points

        theta_history = np.zeros((self.num_time_steps + 1, N))
        theta_history[0, :] = theta

        C = abs(self.U) * self.dt / self.dx # Parameter C for the implicit formula
        divisor = 1.0 + C

        for n in range(self.num_time_steps):
            theta_next = np.zeros_like(theta)

            # Boundary Condition
            theta_next[0] = boundary_theta[n]
            """
            boundary_theta is no long a constant like previous test cases,
            this makes boundary_theta an array of the values over time,
            created by exp_decay_array function
            """

            # Forward Substitution
            for i in range(1, N):
                numerator = theta[i] + C * theta_next[i-1]
                theta_next[i] = numerator / divisor

            theta = theta_next
            theta_history[n+1, :] = theta

        return theta_history

    def compute_solution(self, theta_initial, boundary_theta):

        return self._solve_implicit(theta_initial, boundary_theta)


"""
Test case 4 : exponentially decaying theta
rest are the same as case 1
need 31 values for theta
"""
L = 20
T = 300
dx = 0.2
dt = 10
U = 0.1


def exp_decay_array(length, decay_factor, initial=250):
  arr = np.arange(length)
  decay_array = np.exp(-decay_factor * arr)
  return initial * decay_array

n = int(T/dt) +1
boundary_theta = exp_decay_array(n, decay_factor=0.05, initial=250) #adjust values of decay factor to change the rate of decay, or initial for different cases


solver_A = AdvectionSolver(L, dx, dt, T, U)

theta_initial = np.zeros_like(solver_A.x_new)
theta_initial[0] = boundary_theta[0]
res_A = solver_A.compute_solution(theta_initial, boundary_theta)

#Create animation
fig, ax = plt.subplots(figsize=(4,3))
line, = ax.plot([], [], lw=2)

ax.set_xlim(0, L)
ax.set_ylim(0, 300)
ax.set_xlabel("x (m)", fontsize=8)
ax.set_ylabel("Concentration (µg/m³)", fontsize=8)

def init():
  line.set_data([], [] )
  return line,

def animate(t):
  y = res_A[t]
  line.set_data(solver_A.x_new, y)
  ax.set_title("Concentration of pollutant at t = " + str(dt*t) + "s", fontsize=10, verticalalignment='top')
  return line,


anim = matplotlib.animation.FuncAnimation(fig, animate, init_func=init, frames=len(res_A), blit = True)
plt.close(fig)
# to prevent a second graph of t=300s
display(HTML(anim.to_jshtml()))


Using **Implicit Scheme (Standard BTBS)**.
